In [ ]:
import mmengine
import numpy as np
from mmengine.fileio import get
from mmdet3d.structures import LiDARInstance3DBoxes

In [ ]:
info = mmengine.load('./data/JRDB/JRDB_infos_train_small.pkl')

In [ ]:
info.keys()

In [ ]:
info['metainfo']

In [ ]:
len(info['data_list']), info['data_list'][0].keys()

In [ ]:
info['data_list'][0]['images'].keys(), info['data_list'][0]['images']['cam0'].keys()

In [ ]:
info['data_list'][0]['lidar_points'].keys()

In [ ]:
len(info['data_list'][0]['instances'][0]), info['data_list'][0]['instances'][0].keys()

In [ ]:
bboxes_3d = [instance['bbox_3d'] for instance in info['data_list'][0]['instances']]

In [ ]:
bboxes_3d_ = LiDARInstance3DBoxes(bboxes_3d)

In [ ]:
bboxes_3d

In [ ]:
bboxes_3d_

In [ ]:
# 计算标签的范围
min = np.array([np.inf, np.inf, np.inf, np.inf, np.inf, np.inf])
max = np.array([-np.inf, -np.inf, -np.inf, -np.inf, -np.inf, -np.inf])
count = 0
for data in info['data_list']:
    for instence in data['instances']:
        min = np.minimum(min, instence['bbox_3d'][:6])
        max = np.maximum(max, instence['bbox_3d'][:6])
        count += 1
min, max, count

In [ ]:
i, j = 1, 1
bbox_3d = info['data_list'][i]['instances'][j]['bbox_3d']
bbox = info['data_list'][i]['instances'][j]['bbox']
lidar2img = info['data_list'][i]['images']['left']['lidar2img']
bbox_3d, bbox, lidar2img

In [ ]:
info = info['data_list'][4]
lidar2img = np.array(info['images']['left']['cam2img']) @ np.array(info['images']['left']['lidar2cam'])

In [ ]:
lidar2img.tolist()

In [1]:
import yaml
import os
import numpy as np
import cv2
import open3d as o3d
import matplotlib.pyplot as plt
import mmengine

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [ ]:
def undisort_image(image, K, D):
    h, w = image.shape[:2]
    new_K, roi = cv2.getOptimalNewCameraMatrix(K, D, (w, h), 0, (w, h))
    undistorted_image = cv2.undistort(image, K, D, None, new_K)
    return undistorted_image, new_K

data_dir = './data/JRDB/'
calibration_dir = "../jrdb_toolkit/calibration/"
location = 'stlc-111-2019-04-19_0'
lidar_config = os.path.join(calibration_dir, "lidars.yaml")

with open(lidar_config, 'r') as f:
    lidar_calib = yaml.safe_load(f)

idx = 1500
infos = mmengine.load('./data/JRDB/JRDB_infos_train_small.pkl')
info = infos['data_list'][idx]

# Load pointclouds    
lower_pcd = o3d.io.read_point_cloud(
    os.path.join(data_dir, info['lidar_points']['lidar_path_lower'])).points
upper_pcd = o3d.io.read_point_cloud(
    os.path.join(data_dir, info['lidar_points']['lidar_path_upper'])).points
# homogenize the points
upper_pcd = np.hstack([np.array(upper_pcd), np.ones((len(upper_pcd), 1))])
lower_pcd = np.hstack([np.array(lower_pcd), np.ones((len(lower_pcd), 1))])
lower2upper = np.array(info["lidar_points"]["lower2upper"])
###### apply the transformation to the lower lidar ########
lower_pcd = np.dot(lower2upper, np.transpose(np.array(lower_pcd)))
# merge the pointclouds
merged_pcd = np.hstack([np.transpose(np.array(upper_pcd)), lower_pcd]).transpose()
bbox_3d = np.array([item['bbox_3d'] for item in info['instances']])
print(bbox_3d.shape)

# Iterating over specified camera IDs to process and visualize point clouds on undistorted images.
for cam_id in [0, 2, 4, 6, 8]:
    # Retrieve calibration data for the current camera, including both distorted and undistorted intrinsic matrices.
    undistorted_K = np.array(info['images']['cam'+str(cam_id)]['cam2img'])[:3, :3]
    undistorted_K = np.hstack([undistorted_K, np.zeros((3, 1))])  # Homogenize the undistorted intrinsic matrix.
    distored_K = np.array(info['images']['cam'+str(cam_id)]['distored_K']).reshape(3, 3)
    D = np.array(info['images']['cam'+str(cam_id)]['D'])  # Distortion coefficients.

    # Compute the transformation matrix from the LiDAR (upper) to the camera coordinate system and project the points.
    upper2ego = np.array(info["lidar_points"]["lidar2ego"])
    ego2cam = np.array(info['images']['cam'+str(cam_id)]["lidar2cam"])
    pts_ref = ego2cam.dot(upper2ego.dot(merged_pcd.T)).T
    pts_ref = undistorted_K.dot(pts_ref.T).T  # Project points using the undistorted intrinsic matrix.
    pts_ref = pts_ref[pts_ref[:, 2] > 0]  # Keep only points in front of the camera.
    pts_ref = pts_ref[:, :2] / pts_ref[:, 2].reshape(-1, 1)  # Normalize points to the image plane.

    # Load the corresponding camera image, undistort it, and plot the projected points on this undistorted image.
    image = cv2.imread(os.path.join(data_dir, info['images']['cam'+str(cam_id)]['img_path']))
    undistorted_image, _ = undisort_image(image, distored_K, D)  # Undistort the image using its intrinsic matrix and distortion coefficients.

    # Draw each point on the undistorted image.
    for pts in pts_ref:
        x, y = pts
        if 0 <= x < undistorted_image.shape[1] and 0 <= y < undistorted_image.shape[0]:
            cv2.circle(undistorted_image, (int(x), int(y)), 1, (0, 255, 0), -1)

    # 将 bbox_3d 投影到图像上，bbox_3d 的格式是 [x, y, z, l, w, h, yaw]
    for bbox in bbox_3d:
        x, y, z, l, w, h, ry = bbox
        # 3D bounding box corners
        corners = np.array([
            [l/2, w/2, h/2],
            [l/2, w/2, -h/2],
            [l/2, -w/2, h/2],
            [l/2, -w/2, -h/2],
            [-l/2, w/2, h/2],
            [-l/2, w/2, -h/2],
            [-l/2, -w/2, h/2],
            [-l/2, -w/2, -h/2],
        ])
        # 3D bounding box rotation matrix
        R = np.array([
            [np.cos(ry), np.sin(ry), 0],
            [-np.sin(ry), np.cos(ry), 0],
            [0, 0, 1]
        ])
        t = np.array([x, y, z+h/2])
        corners = np.dot(R, corners.T).T + t
        corners = np.hstack([corners, np.ones((corners.shape[0], 1))])
        corners = undistorted_K.dot(ego2cam.dot(corners.T)).T
        # 绘制在图像上的 3D bbox 线条
        # for i, j in [(0, 1), (1, 3), (3, 2), (2, 0), (4, 5), (5, 7), (7, 6), (6, 4), (0, 4), (1, 5), (2, 6), (3, 7)]:
        #     if corners[i, 2] > 0 and corners[j, 2] > 0:
        #         cv2.line(undistorted_image, (int(corners[i, 0]/corners[i, 2]), int(corners[i, 1]/corners[i, 2])),
        #                 (int(corners[j, 0]/corners[j, 2]), int(corners[j, 1]/corners[j, 2])), (0, 0, 255), 2)


    # Display the undistorted image with projected points, enhancing visualization for applications like deep learning pipelines.
    plt.figure(figsize=(10, 5))
    plt.imshow(cv2.cvtColor(undistorted_image, cv2.COLOR_BGR2RGB))  # Display the image in RGB format.
    plt.title(f'Camera {cam_id} Point Cloud Projection on Undistorted Image')
    plt.axis('off')  # Hide the axis for a cleaner view.
    plt.show()

In [ ]:
info = mmengine.load('./data/STCrowd/STCrowd_infos_train.pkl')
info2 = mmengine.load('./data/kitti/kitti_infos_trainval2.pkl')

In [ ]:
info_fusion = mmengine.load('./data/kitti_stc_fusion_infos_train.pkl')

In [ ]:
count1 = [0, 0, 0, 0]
count2 = [0, 0, 0]
difficults = [0, 0, 0]
for data in info_fusion['data_list']:
    for instance in data['instances']:
        count1[instance['occluded']] += 1
        difficult = instance['occluded']
        dist = np.sqrt(instance['bbox_3d'][0]**2 + instance['bbox_3d'][1]**2)
        if dist > 20:
            count2[2] += 1
            difficult += 2
        elif dist > 10:
            count2[1] += 1
            difficult += 1
        else:
            count2[0] += 1
        difficults[difficult//2] += 1
count1, count2, difficults

In [ ]:
count = [0, 0, 0]
for data in info_fusion['data_list']:
    for instance in data['instances']:
        count[instance['difficulty']] += 1
count

In [ ]:
([32440, 54602, 48117, 54], [66122, 39256, 29835], [57942, 60957, 16314])

In [ ]:
len(info_fusion['data_list'])

In [ ]:
info_fusion['metainfo'], info_fusion['data_list'][0].keys()

In [ ]:
info2['data_list'][2]['instances']

In [ ]:
instance_min = np.array([np.inf, np.inf, np.inf, np.inf, np.inf, np.inf])
instance_max = np.array([-np.inf, -np.inf, -np.inf, -np.inf, -np.inf, -np.inf])
count = 0
for data in info_fusion['data_list']:
    for instance in data['instances']:
        instance['bbox_3d'] = np.array(instance['bbox_3d'])
        instance_min = np.minimum(instance_min, instance['bbox_3d'][:6])
        instance_max = np.maximum(instance_max, instance['bbox_3d'][:6])
        # if 0 < instance['bbox_3d'][0] < 36 and -18 < instance['bbox_3d'][1] < 18 and -5 < instance['bbox_3d'][2] < 1:
        count += 1
            
instance_min, instance_max, count

In [4]:
noise_rot = np.random.normal(scale=[1, 2, 3], size=3)

In [6]:
noise_rot.shape

(3,)

In [7]:
from scipy.spatial.transform import Rotation

In [10]:
Rotation.from_euler('xyz', noise_rot, degrees=True).as_matrix()

array([[ 0.99929137, -0.00648181,  0.03707754],
       [ 0.00571193,  0.99976666,  0.02083254],
       [-0.03720392, -0.02060599,  0.99909522]])